# <center>MobileNet - Pytorch

# Step 1: Prepare data

In [1]:
# MobileNet-Pytorch
import argparse 
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torchvision import datasets, transforms
from torch.autograd import Variable
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.metrics import accuracy_score
from mobilenets import mobilenet

use_cuda = torch.cuda.is_available()
use_cudause_cud  = torch.cuda.is_available()
dtype = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor

In [2]:
# Train, Validate, Test. Heavily inspired by Kevinzakka https://github.com/kevinzakka/DenseNet/blob/master/data_loader.py

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

valid_size=0.1

# define transforms
valid_transform = transforms.Compose([
        transforms.ToTensor(),
        normalize
])

train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize
])


# load the dataset
train_dataset = datasets.CIFAR10(root="data", train=True, 
            download=True, transform=train_transform)

valid_dataset = datasets.CIFAR10(root="data", train=True, 
            download=True, transform=valid_transform)

num_train = len(train_dataset)
indices = list(range(num_train))
split = int(np.floor(valid_size * num_train)) #5w张图片的10%用来当做验证集


np.random.seed(42)
np.random.shuffle(indices) # 随机乱序[0,1,...,49999]

train_idx, valid_idx = indices[split:], indices[:split]


train_sampler = SubsetRandomSampler(train_idx) # 这个很有意思
valid_sampler = SubsetRandomSampler(valid_idx)


# ------------------------- 使用不同的批次大小 ------------------------------------
show_step=10  # 批次大，show_step就小点
max_epoch=60  # 训练最大epoch数目
train_loader = torch.utils.data.DataLoader(train_dataset, 
                batch_size=256, sampler=train_sampler)

valid_loader = torch.utils.data.DataLoader(valid_dataset, 
                batch_size=256, sampler=valid_sampler)


test_transform = transforms.Compose([
    transforms.ToTensor(), normalize
])

test_dataset = datasets.CIFAR10(root="data", 
                                train=False, 
                                download=True,transform=test_transform)

test_loader = torch.utils.data.DataLoader(test_dataset, 
                                          batch_size=64, 
                                          shuffle=True)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


# Step 2: Model Config

In [3]:
# 32  缩放5次到 1x1@1024 
# From https://github.com/kuangliu/pytorch-cifar 
import torch
import torch.nn as nn
import torch.nn.functional as F


class Block(nn.Module):
    '''Depthwise conv + Pointwise conv'''
    def __init__(self, in_planes, out_planes, stride=1):
        super(Block, self).__init__()
        
        # 分组卷积数=输入通道数
        self.conv1 = nn.Conv2d(in_planes, in_planes, kernel_size=3, stride=stride, padding=1, groups=in_planes, bias=False)
        
        self.bn1 = nn.BatchNorm2d(in_planes)
        
        self.conv2 = nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=1, padding=0, bias=False)
        
        self.bn2 = nn.BatchNorm2d(out_planes)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        return out


class MobileNet(nn.Module):
    # (128,2) means conv planes=128, conv stride=2, by default conv stride=1
    cfg = [64, (128,2), 128, (256,2), 256, (512,2), 512, 512, 512, 512, 512, (1024,2), 1024]

    def __init__(self, num_classes=10):
        super(MobileNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False)
        
        self.bn1 = nn.BatchNorm2d(32)
        self.layers = self._make_layers(in_planes=32) # 自动化构建层
        self.linear = nn.Linear(1024, num_classes)

    def _make_layers(self, in_planes):
        layers = []
        for x in self.cfg:
            out_planes = x if isinstance(x, int) else x[0]
            stride = 1 if isinstance(x, int) else x[1]
            layers.append(Block(in_planes, out_planes, stride))
            in_planes = out_planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layers(out)
        out = F.avg_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [4]:
# From https://github.com/Z0m6ie/CIFAR-10_PyTorch
#model = mobilenet(num_classes=10, large_img=False)

# From https://github.com/kuangliu/pytorch-cifar 
model=MobileNet(10).cuda()

optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = StepLR(optimizer, step_size=10, gamma=0.5)
criterion = nn.CrossEntropyLoss()

In [5]:
# Implement validation
def train(epoch):
    model.train()
    #writer = SummaryWriter()
    for batch_idx, (data, target) in enumerate(train_loader):
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        correct = 0
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).sum()
        
        loss = criterion(output, target)
        loss.backward()
        accuracy = 100. * (correct.cpu().numpy()/ len(output))
        optimizer.step()
        if batch_idx % show_step == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}, Accuracy: {:.2f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item(), accuracy))
            #writer.add_scalar('Loss/Loss', loss.item(), epoch)
            #writer.add_scalar('Accuracy/Accuracy', accuracy, epoch)
    scheduler.step()

In [6]:
def validate(epoch):
    model.eval()
    #writer = SummaryWriter()
    valid_loss = 0
    correct = 0
    for data, target in valid_loader:
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        output = model(data)
        valid_loss += F.cross_entropy(output, target, size_average=False).item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).sum()

    valid_loss /= len(valid_idx)
    accuracy = 100. * correct.cpu().numpy() / len(valid_idx)
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        valid_loss, correct, len(valid_idx),
        100. * correct / len(valid_idx)))
    #writer.add_scalar('Loss/Validation_Loss', valid_loss, epoch)
    #writer.add_scalar('Accuracy/Validation_Accuracy', accuracy, epoch)
    return valid_loss, accuracy

In [7]:
# Fix best model

def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        output = model(data)
        test_loss += F.cross_entropy(output, target, size_average=False).item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct.cpu().numpy() / len(test_loader.dataset)))

In [8]:
def save_best(loss, accuracy, best_loss, best_acc):
    if best_loss == None:
        best_loss = loss
        best_acc = accuracy
        file = 'saved_models/best_save_model.p'
        torch.save(model.state_dict(), file)
        
    elif loss < best_loss and accuracy > best_acc:
        best_loss = loss
        best_acc = accuracy
        file = 'saved_models/best_save_model.p'
        torch.save(model.state_dict(), file)
    return best_loss, best_acc

In [9]:
# Fantastic logger for tensorboard and pytorch, 
# run tensorboard by opening a new terminal and run "tensorboard --logdir runs"
# open tensorboard at http://localhost:6006/
from tensorboardX import SummaryWriter
best_loss = None
best_acc = None

import time 
SINCE=time.time()

for epoch in range(max_epoch):
    train(epoch)
    loss, accuracy = validate(epoch)
    best_loss, best_acc = save_best(loss, accuracy, best_loss, best_acc)
    
    NOW=time.time() 
    DURINGS=NOW-SINCE
    SINCE=NOW
    print("the time of this epoch:[{} s]".format(DURINGS))
    
#writer = SummaryWriter()
#writer.export_scalars_to_json("./all_scalars.json")

#writer.close()

#---------------------------- Test ------------------------------
test(epoch)

Train Epoch: 0 [0/50000 (0%)]	Loss: 2.326184, Accuracy: 7.42
Train Epoch: 0 [2560/50000 (6%)]	Loss: 2.547386, Accuracy: 13.67
Train Epoch: 0 [5120/50000 (11%)]	Loss: 2.176494, Accuracy: 19.92
Train Epoch: 0 [7680/50000 (17%)]	Loss: 1.995289, Accuracy: 23.83
Train Epoch: 0 [10240/50000 (23%)]	Loss: 1.929524, Accuracy: 23.83
Train Epoch: 0 [12800/50000 (28%)]	Loss: 1.874008, Accuracy: 27.73
Train Epoch: 0 [15360/50000 (34%)]	Loss: 1.845196, Accuracy: 26.17
Train Epoch: 0 [17920/50000 (40%)]	Loss: 1.734499, Accuracy: 30.08
Train Epoch: 0 [20480/50000 (45%)]	Loss: 1.629688, Accuracy: 33.59
Train Epoch: 0 [23040/50000 (51%)]	Loss: 1.766238, Accuracy: 32.03
Train Epoch: 0 [25600/50000 (57%)]	Loss: 1.803599, Accuracy: 30.08
Train Epoch: 0 [28160/50000 (62%)]	Loss: 1.792998, Accuracy: 33.98
Train Epoch: 0 [30720/50000 (68%)]	Loss: 1.699838, Accuracy: 31.64
Train Epoch: 0 [33280/50000 (74%)]	Loss: 1.720951, Accuracy: 31.25
Train Epoch: 0 [35840/50000 (80%)]	Loss: 1.669409, Accuracy: 39.84
Train

Train Epoch: 6 [12800/50000 (28%)]	Loss: 0.657822, Accuracy: 76.95
Train Epoch: 6 [15360/50000 (34%)]	Loss: 0.647336, Accuracy: 78.52
Train Epoch: 6 [17920/50000 (40%)]	Loss: 0.632608, Accuracy: 79.30
Train Epoch: 6 [20480/50000 (45%)]	Loss: 0.608753, Accuracy: 78.91
Train Epoch: 6 [23040/50000 (51%)]	Loss: 0.672278, Accuracy: 75.00
Train Epoch: 6 [25600/50000 (57%)]	Loss: 0.646113, Accuracy: 75.78
Train Epoch: 6 [28160/50000 (62%)]	Loss: 0.577857, Accuracy: 80.08
Train Epoch: 6 [30720/50000 (68%)]	Loss: 0.722544, Accuracy: 77.73
Train Epoch: 6 [33280/50000 (74%)]	Loss: 0.681466, Accuracy: 78.91
Train Epoch: 6 [35840/50000 (80%)]	Loss: 0.637329, Accuracy: 80.08
Train Epoch: 6 [38400/50000 (85%)]	Loss: 0.610807, Accuracy: 79.30
Train Epoch: 6 [40960/50000 (91%)]	Loss: 0.527092, Accuracy: 84.77
Train Epoch: 6 [43520/50000 (97%)]	Loss: 0.700928, Accuracy: 76.17

Validation set: Average loss: 0.9283, Accuracy: 3567/5000 (71.00%)

the time of this epoch:[19.741650104522705 s]
Train Epoch: 7

Train Epoch: 12 [25600/50000 (57%)]	Loss: 0.362180, Accuracy: 87.11
Train Epoch: 12 [28160/50000 (62%)]	Loss: 0.415848, Accuracy: 83.98
Train Epoch: 12 [30720/50000 (68%)]	Loss: 0.367759, Accuracy: 85.55
Train Epoch: 12 [33280/50000 (74%)]	Loss: 0.372398, Accuracy: 87.11
Train Epoch: 12 [35840/50000 (80%)]	Loss: 0.387422, Accuracy: 87.11
Train Epoch: 12 [38400/50000 (85%)]	Loss: 0.353944, Accuracy: 87.89
Train Epoch: 12 [40960/50000 (91%)]	Loss: 0.413531, Accuracy: 84.38
Train Epoch: 12 [43520/50000 (97%)]	Loss: 0.372726, Accuracy: 86.72

Validation set: Average loss: 0.4608, Accuracy: 4213/5000 (84.00%)

the time of this epoch:[19.709614276885986 s]
Train Epoch: 13 [0/50000 (0%)]	Loss: 0.302592, Accuracy: 89.84
Train Epoch: 13 [2560/50000 (6%)]	Loss: 0.299214, Accuracy: 91.02
Train Epoch: 13 [5120/50000 (11%)]	Loss: 0.454557, Accuracy: 86.33
Train Epoch: 13 [7680/50000 (17%)]	Loss: 0.279018, Accuracy: 89.06
Train Epoch: 13 [10240/50000 (23%)]	Loss: 0.323915, Accuracy: 89.84
Train Epoc

Train Epoch: 18 [35840/50000 (80%)]	Loss: 0.323387, Accuracy: 87.11
Train Epoch: 18 [38400/50000 (85%)]	Loss: 0.325529, Accuracy: 85.55
Train Epoch: 18 [40960/50000 (91%)]	Loss: 0.342458, Accuracy: 88.67
Train Epoch: 18 [43520/50000 (97%)]	Loss: 0.330802, Accuracy: 88.67

Validation set: Average loss: 0.4286, Accuracy: 4271/5000 (85.00%)

the time of this epoch:[19.730318069458008 s]
Train Epoch: 19 [0/50000 (0%)]	Loss: 0.268865, Accuracy: 90.62
Train Epoch: 19 [2560/50000 (6%)]	Loss: 0.361111, Accuracy: 86.33
Train Epoch: 19 [5120/50000 (11%)]	Loss: 0.284406, Accuracy: 90.62
Train Epoch: 19 [7680/50000 (17%)]	Loss: 0.266973, Accuracy: 91.41
Train Epoch: 19 [10240/50000 (23%)]	Loss: 0.350080, Accuracy: 88.67
Train Epoch: 19 [12800/50000 (28%)]	Loss: 0.341272, Accuracy: 89.45
Train Epoch: 19 [15360/50000 (34%)]	Loss: 0.302849, Accuracy: 87.89
Train Epoch: 19 [17920/50000 (40%)]	Loss: 0.267279, Accuracy: 89.06
Train Epoch: 19 [20480/50000 (45%)]	Loss: 0.365786, Accuracy: 85.94
Train Epoc


Validation set: Average loss: 0.3743, Accuracy: 4406/5000 (88.00%)

the time of this epoch:[19.66059184074402 s]
Train Epoch: 25 [0/50000 (0%)]	Loss: 0.201792, Accuracy: 93.36
Train Epoch: 25 [2560/50000 (6%)]	Loss: 0.152647, Accuracy: 93.75
Train Epoch: 25 [5120/50000 (11%)]	Loss: 0.206790, Accuracy: 92.97
Train Epoch: 25 [7680/50000 (17%)]	Loss: 0.167408, Accuracy: 94.14
Train Epoch: 25 [10240/50000 (23%)]	Loss: 0.336767, Accuracy: 88.28
Train Epoch: 25 [12800/50000 (28%)]	Loss: 0.121853, Accuracy: 94.53
Train Epoch: 25 [15360/50000 (34%)]	Loss: 0.211726, Accuracy: 93.36
Train Epoch: 25 [17920/50000 (40%)]	Loss: 0.156216, Accuracy: 94.14
Train Epoch: 25 [20480/50000 (45%)]	Loss: 0.234634, Accuracy: 92.97
Train Epoch: 25 [23040/50000 (51%)]	Loss: 0.251925, Accuracy: 91.02
Train Epoch: 25 [25600/50000 (57%)]	Loss: 0.157786, Accuracy: 95.70
Train Epoch: 25 [28160/50000 (62%)]	Loss: 0.199999, Accuracy: 91.41
Train Epoch: 25 [30720/50000 (68%)]	Loss: 0.284165, Accuracy: 91.80
Train Epoch

Train Epoch: 31 [5120/50000 (11%)]	Loss: 0.135843, Accuracy: 94.53
Train Epoch: 31 [7680/50000 (17%)]	Loss: 0.153409, Accuracy: 93.75
Train Epoch: 31 [10240/50000 (23%)]	Loss: 0.230801, Accuracy: 91.02
Train Epoch: 31 [12800/50000 (28%)]	Loss: 0.127163, Accuracy: 94.92
Train Epoch: 31 [15360/50000 (34%)]	Loss: 0.130160, Accuracy: 95.70
Train Epoch: 31 [17920/50000 (40%)]	Loss: 0.193209, Accuracy: 93.75
Train Epoch: 31 [20480/50000 (45%)]	Loss: 0.138165, Accuracy: 93.36
Train Epoch: 31 [23040/50000 (51%)]	Loss: 0.176195, Accuracy: 94.53
Train Epoch: 31 [25600/50000 (57%)]	Loss: 0.128100, Accuracy: 96.09
Train Epoch: 31 [28160/50000 (62%)]	Loss: 0.173164, Accuracy: 94.53
Train Epoch: 31 [30720/50000 (68%)]	Loss: 0.132044, Accuracy: 95.31
Train Epoch: 31 [33280/50000 (74%)]	Loss: 0.133763, Accuracy: 94.92
Train Epoch: 31 [35840/50000 (80%)]	Loss: 0.114083, Accuracy: 95.70
Train Epoch: 31 [38400/50000 (85%)]	Loss: 0.139701, Accuracy: 94.92
Train Epoch: 31 [40960/50000 (91%)]	Loss: 0.200573

Train Epoch: 37 [15360/50000 (34%)]	Loss: 0.132684, Accuracy: 94.92
Train Epoch: 37 [17920/50000 (40%)]	Loss: 0.127480, Accuracy: 94.92
Train Epoch: 37 [20480/50000 (45%)]	Loss: 0.116871, Accuracy: 96.09
Train Epoch: 37 [23040/50000 (51%)]	Loss: 0.131617, Accuracy: 96.48
Train Epoch: 37 [25600/50000 (57%)]	Loss: 0.168775, Accuracy: 93.75
Train Epoch: 37 [28160/50000 (62%)]	Loss: 0.112142, Accuracy: 95.31
Train Epoch: 37 [30720/50000 (68%)]	Loss: 0.103440, Accuracy: 95.31
Train Epoch: 37 [33280/50000 (74%)]	Loss: 0.116958, Accuracy: 95.31
Train Epoch: 37 [35840/50000 (80%)]	Loss: 0.048390, Accuracy: 98.05
Train Epoch: 37 [38400/50000 (85%)]	Loss: 0.100047, Accuracy: 96.09
Train Epoch: 37 [40960/50000 (91%)]	Loss: 0.168268, Accuracy: 94.14
Train Epoch: 37 [43520/50000 (97%)]	Loss: 0.166038, Accuracy: 94.53

Validation set: Average loss: 0.4110, Accuracy: 4439/5000 (88.00%)

the time of this epoch:[19.48387885093689 s]
Train Epoch: 38 [0/50000 (0%)]	Loss: 0.108833, Accuracy: 96.88
Train E

Train Epoch: 43 [25600/50000 (57%)]	Loss: 0.098453, Accuracy: 97.27
Train Epoch: 43 [28160/50000 (62%)]	Loss: 0.076608, Accuracy: 98.05
Train Epoch: 43 [30720/50000 (68%)]	Loss: 0.064481, Accuracy: 97.27
Train Epoch: 43 [33280/50000 (74%)]	Loss: 0.085159, Accuracy: 97.27
Train Epoch: 43 [35840/50000 (80%)]	Loss: 0.058158, Accuracy: 97.66
Train Epoch: 43 [38400/50000 (85%)]	Loss: 0.078399, Accuracy: 97.27
Train Epoch: 43 [40960/50000 (91%)]	Loss: 0.112095, Accuracy: 95.70
Train Epoch: 43 [43520/50000 (97%)]	Loss: 0.070004, Accuracy: 97.66

Validation set: Average loss: 0.4188, Accuracy: 4455/5000 (89.00%)

the time of this epoch:[19.474637269973755 s]
Train Epoch: 44 [0/50000 (0%)]	Loss: 0.101173, Accuracy: 96.88
Train Epoch: 44 [2560/50000 (6%)]	Loss: 0.055456, Accuracy: 97.66
Train Epoch: 44 [5120/50000 (11%)]	Loss: 0.103427, Accuracy: 96.48
Train Epoch: 44 [7680/50000 (17%)]	Loss: 0.099214, Accuracy: 96.09
Train Epoch: 44 [10240/50000 (23%)]	Loss: 0.055708, Accuracy: 98.05
Train Epoc

Train Epoch: 49 [35840/50000 (80%)]	Loss: 0.047215, Accuracy: 97.27
Train Epoch: 49 [38400/50000 (85%)]	Loss: 0.077649, Accuracy: 97.66
Train Epoch: 49 [40960/50000 (91%)]	Loss: 0.094034, Accuracy: 96.09
Train Epoch: 49 [43520/50000 (97%)]	Loss: 0.079857, Accuracy: 97.66

Validation set: Average loss: 0.4365, Accuracy: 4444/5000 (88.00%)

the time of this epoch:[19.48315978050232 s]
Train Epoch: 50 [0/50000 (0%)]	Loss: 0.114969, Accuracy: 95.31
Train Epoch: 50 [2560/50000 (6%)]	Loss: 0.055008, Accuracy: 97.27
Train Epoch: 50 [5120/50000 (11%)]	Loss: 0.072173, Accuracy: 97.27
Train Epoch: 50 [7680/50000 (17%)]	Loss: 0.087628, Accuracy: 97.27
Train Epoch: 50 [10240/50000 (23%)]	Loss: 0.049681, Accuracy: 97.66
Train Epoch: 50 [12800/50000 (28%)]	Loss: 0.035249, Accuracy: 98.83
Train Epoch: 50 [15360/50000 (34%)]	Loss: 0.032725, Accuracy: 99.22
Train Epoch: 50 [17920/50000 (40%)]	Loss: 0.037812, Accuracy: 98.05
Train Epoch: 50 [20480/50000 (45%)]	Loss: 0.063303, Accuracy: 98.05
Train Epoch


Validation set: Average loss: 0.4534, Accuracy: 4466/5000 (89.00%)

the time of this epoch:[19.69875741004944 s]
Train Epoch: 56 [0/50000 (0%)]	Loss: 0.050938, Accuracy: 98.44
Train Epoch: 56 [2560/50000 (6%)]	Loss: 0.050141, Accuracy: 97.66
Train Epoch: 56 [5120/50000 (11%)]	Loss: 0.061931, Accuracy: 98.44
Train Epoch: 56 [7680/50000 (17%)]	Loss: 0.050336, Accuracy: 98.05
Train Epoch: 56 [10240/50000 (23%)]	Loss: 0.040370, Accuracy: 98.44
Train Epoch: 56 [12800/50000 (28%)]	Loss: 0.084689, Accuracy: 97.27
Train Epoch: 56 [15360/50000 (34%)]	Loss: 0.099152, Accuracy: 96.88
Train Epoch: 56 [17920/50000 (40%)]	Loss: 0.099631, Accuracy: 96.88
Train Epoch: 56 [20480/50000 (45%)]	Loss: 0.088131, Accuracy: 97.27
Train Epoch: 56 [23040/50000 (51%)]	Loss: 0.065707, Accuracy: 98.05
Train Epoch: 56 [25600/50000 (57%)]	Loss: 0.037553, Accuracy: 98.44
Train Epoch: 56 [28160/50000 (62%)]	Loss: 0.070709, Accuracy: 98.44
Train Epoch: 56 [30720/50000 (68%)]	Loss: 0.037431, Accuracy: 98.44
Train Epoch

# Step 3: Test

In [12]:
test(epoch)


Test set: Average loss: 0.4542, Accuracy: 8932/10000 (89.32%)

